# Summary table for *Adult* data

Here we extract the summary measures for *Adult* data for four classifiers: (1) baseline, (2) project, (3) reduction, and (4) SenSR.
For each of the classifiers we perform gradient flow attack with step size $\frac{1}{100\times i^{2/3}},$ where $i$ is the current step number. We perform 200 steps of attacks on each data points. 

First, we load the summary data for all experiments, and compile it in a panda dataframe. 

In [1]:
import re
import pandas as pd
with open('summary.out', 'r') as f:
    data = f.read()
entries = re.split('\n', data)[:-1]
entries_dict = [eval(e) for e in entries]
data = pd.DataFrame(entries_dict)
data['reject-entropy-loss'] = data['pval'] < 0.05

In [2]:
lr = 1e-2
data_lr = data.loc[data['lr'] == lr]
data_lr = data_lr.rename(columns= {'average_odds_difference_gen': 'AOD-gen', 'average_odds_difference_race': 'AOD-race', 'statistical_parity_difference_gen': 'SPD-gen', 'statistical_parity_difference_race': 'SPD-race', 'equal_opportunity_difference_race': 'EOD-race', 'equal_opportunity_difference_gen': 'EOD-gen', 'lb': 'lb-entropy-loss'})

We only extract the following measures: balanced accuracy, average odds difference for gender and race and lower bound and proportion of times the test being rejected. 

In [3]:
measure = ['bal_acc', 'AOD-gen', 'AOD-race', 'lb-entropy-loss', 'reject-entropy-loss']
agg_dict = dict()
for key in measure:
    if key == 'reject-entropy-loss':
        agg_dict[key] = ['mean']
    else:
        agg_dict[key] = ['mean', 'std']
result = data_lr.groupby(['algo'], as_index=False).agg(agg_dict)
result['algo'] = ['baseline', 'project', 'reduction', 'sensr']

In [4]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import iqr
with open('summary-all.txt', 'r') as f:
    data = f.read()
entries = re.split('\n', data)[:-1]
entries_dict = []
for e in entries:
    try:
        entries_dict.append(eval(e))
    except:
        continue
data = pd.DataFrame(entries_dict)
data = data.loc[data['step'] == 1000]


summary_vars = ['sum-ratio', 'sum-sq-ratio', 'sample-size-ratios', 'sum-start', 'sum-end', 'sum-cov', 'sample-size-0-1']
s_dict = dict()
for s in summary_vars:
    s_dict[s] = {'sum'}


summary = data.groupby(['lr', 'expt', 'step', 'seed-data']).agg(s_dict)


summary['start-mean'] = 1-summary['sum-start']['sum']/summary['sample-size-0-1']['sum']
summary['end-mean'] = 1-summary['sum-end']['sum']/summary['sample-size-0-1']['sum']
summary['cov'] = summary['sum-cov']['sum']/summary['sample-size-0-1']['sum'] + summary['start-mean']\
     + summary['end-mean'] - 1 - summary['start-mean'] * summary['end-mean']
summary['start-sd'] = np.sqrt(summary['start-mean'] * (1-summary['start-mean']))
summary['end-sd'] = np.sqrt(summary['end-mean'] * (1-summary['end-mean']))
summary['n'] = summary['sample-size-0-1']['sum']
summary = summary[['start-mean', 'start-sd', 'end-mean', 'end-sd', 'cov', 'n']]
summary['ratio'] = summary['end-mean'] / summary['start-mean']
summary['ratio-sd'] = np.sqrt((summary['start-mean']**2 * summary['end-sd']**2 \
    + summary['start-sd']**2 * summary['end-mean']**2\
         - 2 * summary['cov'] * summary['start-mean'] * summary['end-mean'])\
             /(summary['n'] * summary['start-mean']**4))

summary['lb-01-loss'] = summary['ratio'] - 1.64 * summary['ratio-sd']
summary['reject-01-loss'] = summary['lb-01-loss'] > 1.25
summary['algo'] = ['baseline'] * 10 + ['project'] * 10 + ['reduction'] * 10 + ['sensr'] * 10
summary_type = {'lb-01-loss': ['mean', 'std'], 'reject-01-loss': ['mean']}
summary = summary.groupby(['algo']).agg(summary_type)

In [5]:
summary = pd.merge(result, summary, left_on='algo', right_on='algo')
rows = ['sensr', 'reduction', 'baseline', 'project']
row_names = ['Baseline', 'Project', 'Reduction','SenSR']
colnames=['Balanced Acc', '$\\text{AOD}_{\\text{gen}}$', '$\\text{AOD}_{\\text{race}}$', '$T_n$', 'Rejection Prop']

for i, c in enumerate(measure):
    if c == 'reject-entropy-loss':
        idx_best = (1-result[c]['mean']).idxmin()
        result[colnames[i]] = (result[c]['mean']).apply('{:.1f}'.format)
        #result.at[idx_best, colnames[i]] = '\\textbf{' + result[colnames[i]][idx_best] + '}'
        
    elif c == 'bal_acc':
        idx_best = (result[c]['mean']).idxmax()
        result[colnames[i]] = result[c]['mean'].apply("{:.3f}".format)+ '$\pm$' + result[c]['std'].apply("{:.3f}".format)
        best_val = result[colnames[i]][idx_best].split('$\pm$')
        best_val = '$\pm$'.join(['\\textbf{' + best_val[0] + '}', best_val[1]])
        #result.at[idx_best, colnames[i]] = best_val
    elif c == 'lb-entropy-loss':
        idx_best = (result[c]['mean']).idxmin()
        result[colnames[i]] = result[c]['mean'].apply("{:.3f}".format)+ '$\pm$' + result[c]['std'].apply("{:.3f}".format)
        best_val = result[colnames[i]][idx_best].split('$\pm$')
        best_val = '$\pm$'.join(['\\textbf{' + best_val[0] + '}', best_val[1]])
        #result.at[idx_best, colnames[i]] = best_val
    else:
        idx_best = (result[c]['mean']).abs().idxmin()
        result[colnames[i]] = result[c]['mean'].apply("{:.3f}".format)+ '$\pm$' + result[c]['std'].apply("{:.3f}".format)
        best_val = result[colnames[i]][idx_best].split('$\pm$')
        best_val = '$\pm$'.join(['\\textbf{' + best_val[0] + '}', best_val[1]])
        #result.at[idx_best, colnames[i]] = best_val
ind = dict()
for i, expt in enumerate(row_names):
    ind[i] = expt
    res = result[colnames].rename(index=ind)

res = res.reindex(['SenSR', 'Reduction', 'Project', 'Baseline'])
res

,Balanced Acc,$\text{AOD}_{\text{gen}}$,$\text{AOD}_{\text{race}}$,$T_n$,Rejection Prop
,,,,,
SenSR,0.765$\pm$0.012,-0.074$\pm$0.033,-0.048$\pm$0.008,1.021$\pm$0.008,0.0
Reduction,0.800$\pm$0.005,0.001$\pm$0.021,-0.027$\pm$0.013,5.712$\pm$2.264,1.0
Project,0.825$\pm$0.003,-0.147$\pm$0.015,-0.053$\pm$0.015,1.660$\pm$0.355,0.9
Baseline,0.817$\pm$0.007,-0.151$\pm$0.026,-0.061$\pm$0.015,3.676$\pm$2.164,1.0


In [6]:
from itertools import product

index = summary.T.index
index_subset = [index[i] for i in [1, 3, 5, 7, 9, 10, 12]]
summary_mean = summary[index_subset]
new_index = [( '', 'bal-acc'),('', '$\\text{AOD}_{\\text{gen}}$'),\
    ('', '$\\text{AOD}_{\\text{race}}$'),\
     ('Entropy loss', '$T_n$'), ('Entropy loss', 'rejection prop'),\
          ('0-1 loss', '$T_n$'), ('0-1 loss', 'rejection prop')] 
summary_mean = summary_mean.astype('float32')

for i, j in zip(index_subset, new_index):
    if  i == ('reject-entropy-loss', 'mean') or i == ('reject-01-loss', 'mean'):
        idx_best = (summary_mean[i]).idxmin()
        summary_mean[j] = (summary_mean[i]).apply('{:.1f}'.format)
        summary_mean.at[idx_best, j] = '\\textbf{' + summary_mean[j][idx_best] + '}'

    elif i == ('bal_acc', 'mean'):
        idx_best = (summary_mean[i]).idxmax()
        summary_mean[j] = summary_mean[i].apply("{:.3f}".format)
        best_val = summary_mean[j][idx_best]
        best_val = '\\textbf{' + best_val + '}'
        summary_mean.at[idx_best, j] = best_val
    elif i == ('lb-entropy-loss', 'mean') or i == ('lb-01-loss', 'mean'):
        idx_best = (summary_mean[i]).idxmin()
        summary_mean[j] = summary_mean[i].apply("{:.3f}".format)
        best_val = summary_mean[j][idx_best]
        best_val = '\\textbf{' + best_val + '}'
        summary_mean.at[idx_best, j] = best_val
    else:
        idx_best = (summary_mean[i]).abs().idxmin()
        summary_mean[j] = summary_mean[i].apply("{:.3f}".format)
        best_val = summary_mean[j][idx_best]
        best_val = '\\textbf{' + best_val + '}'
        summary_mean.at[idx_best, j] = best_val
summary_mean = summary_mean[new_index]
rows = ['sensr', 'reduction', 'baseline', 'project']
row_names = ['Baseline', 'Project', 'Reduction','SenSR']
ind = dict()
for i, expt in enumerate(row_names):
    ind[i] = expt
summary_mean = summary_mean.rename(index=ind)
summary_mean

\
                  bal-acc $\text{AOD}_{\text{gen}}$   
Baseline            0.817                    -0.151   
Project    \textbf{0.825}                    -0.147   
Reduction           0.800            \textbf{0.001}   
SenSR               0.765                    -0.074   

                                        Entropy loss                 \
          $\text{AOD}_{\text{race}}$           $T_n$ rejection prop   
Baseline                      -0.061           3.676            1.0   
Project                       -0.053           1.660            0.9   
Reduction            \textbf{-0.027}           5.712            1.0   
SenSR                         -0.048  \textbf{1.021}   \textbf{0.0}   

                 0-1 loss                 
                    $T_n$ rejection prop  
Baseline            2.262            1.0  
Project             1.800            0.8  
Reduction           3.275            1.0  
SenSR      \textbf{1.081}   \textbf{0.0}

In [7]:
print(summary_mean.to_latex(escape=False, index=True))

\begin{tabular}{llllllll}
\toprule
{} & \multicolumn{2}{l}{Entropy loss} & \multicolumn{2}{l}{0-1 loss} \\
{} &         bal-acc & $\text{AOD}_{\text{gen}}$ & $\text{AOD}_{\text{race}}$ &           $T_n$ & rejection prop &           $T_n$ & rejection prop \\
\midrule
Baseline  &           0.817 &                    -0.151 &                     -0.061 &           3.676 &            1.0 &           2.262 &            1.0 \\
Project   &  \textbf{0.825} &                    -0.147 &                     -0.053 &           1.660 &            0.9 &           1.800 &            0.8 \\
Reduction &           0.800 &            \textbf{0.001} &            \textbf{-0.027} &           5.712 &            1.0 &           3.275 &            1.0 \\
SenSR     &           0.765 &                    -0.074 &                     -0.048 &  \textbf{1.021} &   \textbf{0.0} &  \textbf{1.081} &   \textbf{0.0} \\
\bottomrule
\end{tabular}



In [8]:
for i, j in zip(index_subset, new_index):
    k = i[0], 'std'
    if  i == ('reject-entropy-loss', 'mean') or i == ('reject-01-loss', 'mean'):
        idx_best = (summary[i]).idxmin()
        summary[j] = (summary[i]).apply('{:.1f}'.format)
        summary.at[idx_best, j] = '\\textbf{' + summary[j][idx_best] + '}'

    elif i == ('bal_acc', 'mean'):
        idx_best = (summary[i]).idxmax()
        summary[j] = summary[i].apply("{:.3f}".format)+ '$\pm$' + summary[k].apply("{:.3f}".format)
        best_val = summary[j][idx_best].split('$\pm$')
        best_val = '$\pm$'.join(['\\textbf{' + best_val[0] + '}', best_val[1]])
        summary.at[idx_best, j] = best_val
    elif i == ('lb-entropy-loss', 'mean') or i == ('lb-01-loss', 'mean'):
        idx_best = (summary[i]).idxmin()
        summary[j] = summary[i].apply("{:.3f}".format)+ '$\pm$' + summary[k].apply("{:.3f}".format)
        best_val = summary[j][idx_best].split('$\pm$')
        best_val = '$\pm$'.join(['\\textbf{' + best_val[0] + '}', best_val[1]])
        summary.at[idx_best, j] = best_val
    else:
        idx_best = (summary[i]).abs().idxmin()
        summary[j] = summary[i].apply("{:.3f}".format)+ '$\pm$' + summary[k].apply("{:.3f}".format)
        best_val = summary[j][idx_best].split('$\pm$')
        best_val = '$\pm$'.join(['\\textbf{' + best_val[0] + '}', best_val[1]])
        summary.at[idx_best, j] = best_val

summary = summary[new_index]
rows = ['sensr', 'reduction', 'baseline', 'project']
row_names = ['Baseline', 'Project', 'Reduction','SenSR']
ind = dict()
for i, expt in enumerate(row_names):
    ind[i] = expt
summary = summary.rename(index=ind)
(summary.T)

Baseline  \
             bal-acc                      0.817$\pm$0.007   
             $\text{AOD}_{\text{gen}}$   -0.151$\pm$0.026   
             $\text{AOD}_{\text{race}}$  -0.061$\pm$0.015   
Entropy loss $T_n$                        3.676$\pm$2.164   
             rejection prop                           1.0   
0-1 loss     $T_n$                        2.262$\pm$0.356   
             rejection prop                           1.0   

                                                          Project  \
             bal-acc                     \textbf{0.825}$\pm$0.003   
             $\text{AOD}_{\text{gen}}$           -0.147$\pm$0.015   
             $\text{AOD}_{\text{race}}$          -0.053$\pm$0.015   
Entropy loss $T_n$                                1.660$\pm$0.355   
             rejection prop                                   0.9   
0-1 loss     $T_n$                                1.800$\pm$0.584   
             rejection prop                                   0.8   

                                                         Reduction  \
             bal-acc                               0.800$\pm$0.005   
             $\text{AOD}_{\text{gen}}$    \textbf{0.001}$\pm$0.021   
             $\text{AOD}_{\text{race}}$  \textbf{-0.027}$\pm$0.013   
Entropy loss $T_n$                                 5.712$\pm$2.264   
             rejection prop                                    1.0   
0-1 loss     $T_n$                                 3.275$\pm$0.343   
             rejection prop                                    1.0   

                                                            SenSR  
             bal-acc                              0.765$\pm$0.012  
             $\text{AOD}_{\text{gen}}$           -0.074$\pm$0.033  
             $\text{AOD}_{\text{race}}$          -0.048$\pm$0.008  
Entropy loss $T_n$                       \textbf{1.021}$\pm$0.008  
             rejection prop                          \textbf{0.0}  
0-1 loss     $T_n$                       \textbf{1.081}$\pm$0.041  
             rejection prop                          \textbf{0.0}

In [9]:
print(summary.T.to_latex(escape=False, index=True))

\begin{tabular}{llllll}
\toprule
         &                &          Baseline &                   Project &                  Reduction &                     SenSR \\
\midrule
{} & bal-acc &   0.817$\pm$0.007 &  \textbf{0.825}$\pm$0.003 &            0.800$\pm$0.005 &           0.765$\pm$0.012 \\
         & $\text{AOD}_{\text{gen}}$ &  -0.151$\pm$0.026 &          -0.147$\pm$0.015 &   \textbf{0.001}$\pm$0.021 &          -0.074$\pm$0.033 \\
         & $\text{AOD}_{\text{race}}$ &  -0.061$\pm$0.015 &          -0.053$\pm$0.015 &  \textbf{-0.027}$\pm$0.013 &          -0.048$\pm$0.008 \\
Entropy loss & $T_n$ &   3.676$\pm$2.164 &           1.660$\pm$0.355 &            5.712$\pm$2.264 &  \textbf{1.021}$\pm$0.008 \\
         & rejection prop &               1.0 &                       0.9 &                        1.0 &              \textbf{0.0} \\
0-1 loss & $T_n$ &   2.262$\pm$0.356 &           1.800$\pm$0.584 &            3.275$\pm$0.343 &  \textbf{1.081}$\pm$0.041 \\
         & rejection pro